# Web Scrape labelled images of building styles

The ACOToronto website contains the TOBuilt database -- an open source database of images and information about buildings and structures in toronto.
http://www.acotoronto.ca/tobuilt_new_detailed.php

Order of actions
* we will first use the search tool on the main page to find all the available architectural styles (http://www.acotoronto.ca/tobuilt_new_detailed.php)
* then we will call the page for each style.  This page contains a thumbnail image of each building classified in that style, a link for more details and basic info about the building.  
Note: you generally have to follow a redirection to get to the style page
(http://www.acotoronto.ca/search_buildingsR-d.php?sid=8065)
* download the image locally to image/<style>
* call building details page to get info on building dates, architects etc
(http://www.acotoronto.ca/show_building.php?BuildingID=3883)
 page is structured with alternating building_info and building_info2 divs
building_info contains the name of the info to follow
building_info2 contains the value
The companies row contains the architects. There can be muliple architects for a building, so these are < li > items.
Sometimes the archtect is a hyperlink, but not always, so need to handle both cases specially

American colonial
Annex Style
Art deco
Arts and Crafts
Beaux arts
Brutalist
Byzantine
Chicago style
Classical revival
Commercial style
Contemporary
Deconstructivism
Dutch colonial
Early modern
Edwardian classical
English Cottage style
Georgian revival
Gothic revival
Greek revival
International style
Italianate
Late modernist
Log construction
Mid century expressionist
Mirrored tower
Modern classical
Modern historicist
Modernist
Neo Palladian
Neo-Chateau
Neo-Georgian
Neo-modernist
Neo-Tudor
Postmodern
Prairie style
Queen Anne
Regency
Renaissance revival
Richardsonian Romanesque
Romanesque revival
Sculptural
Second empire
Shingle style
Spanish colonial
Toronto Bay and Gable
Workers Cottage

In [10]:
# Import libaries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import string
import re
import urllib
import os
import time
from datetime import datetime

from selenium import webdriver
from sqlalchemy import sql, Table, MetaData
import ast
from ast import literal_eval
from create_db import connect_db
import pandas as pd

In [11]:
from sqlalchemy import create_engine, Column, Integer, String, Sequence, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from models import connect_db, PointsOfInterest, ArchitecturalStyles, Architects,POICategories

### Set up base variables

In [12]:

main_page = 'http://www.acotoronto.ca/tobuilt_new_detailed.php'
style_url="http://www.acotoronto.ca/search_buildingsDB_d2.php"
site_root = "http://www.acotoronto.ca/"
debug=False
buildings_list=[]
rerun_webscrape=False

df_to_db_map={
    'Name':'name',
    'Completed':'build_year'   ,
    'Demolished' :'demolished_year',
    'Address' :'address' ,
    'Bld_link':'external_url',
    'Notes': 'details',
    'Image':'image_url',
    'Heritage':'heritage_stats',
    'Current use':'current_use',
    'Type':'poi_type'
}

In [ ]:
def load_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Error connecting: status code {response.status_code}")

In [ ]:
def download_image(style, img_url):
    '''
    * downloads image to the appropriate style sub-folder in the images directory (and creates folder if missing)
    * to test: download_image('test','/tobuilt_bk/php/Buildingimages/106BedfordRd.jpg')
    '''

    dest_dir = f"../Images/{style}"

    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    full_url = f'{site_root}/{img_url}'
    image_path = f"{dest_dir}/{img_url.split('/')[-1]}"
    res = urllib.request.urlretrieve(full_url, image_path)


In [ ]:
# borrowed from https://michaeljsanders.com/2017/05/12/scrapin-and-scrollin.html
def get_scrolling_page(page_url):
    browser = webdriver.Chrome("C:\\Users\\blahjays\\Downloads\\chromedriver_win32\\chromedriver.exe")

    # Tell Selenium to get the URL you're interested in.
   # browser.get("http://www.acotoronto.ca/search_buildingsR-d.php?sid=8225")
    browser.get(page_url)

    # Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
            lastCount = lenOfPage
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True

    # Now that the page is fully scrolled, grab the source code.
    source_data = browser.page_source

    # Throw your source into BeautifulSoup and start parsing!
    bs_data = bs(source_data)
    browser.close()
    return bs_data

#### Building Details pages
* page is structured with alternating building_info and building_info2 divs
* building_info contains the name of the info to follow
* building_info2 contains the value
* The companies row contains the architects.  There can be muliple architects for a building, so these are < li > items.
* Sometimes the archtect is a hyperlink, but not always, so need to handle both cases specially

In [ ]:
def get_building_details(bld_link):
    
    flags_dict = {
        'Completed':None,
        'Demolished': None,
        'Companies': None,
        'Type':None,
        'Current use': None,
        'Heritage': None,
       'Notes': None
    }
    architects=[]
    
    full_url = f'{site_root}{bld_link}'
    html_bld = load_page(full_url)
    soup_bld = BeautifulSoup(html_bld)
    # get all building info elements
    info = soup_bld.find_all('div',{'class': 'building_info'})
    for inf in info:
        for flag in flags_dict.keys():
            if flag in inf.text:
                if 'Companies' in flag:
                    company_box = inf.find_next()

                    if 'Architect - ' in company_box.text:
                        li_items = company_box.find_all('li')

                        for arc in li_items:
                            if (arc.find('span')):
                                architect = arc.find('span').previousSibling
                                if architect.name =='a':
                                    architects.append(architect.text.strip())
                                else:
                                    architects.append(architect.replace('-','').strip())
                                
                        flags_dict[flag] = architects
                else:
                    flags_dict[flag] = inf.find_next().text.strip()
    return flags_dict

### Get Buildings for style

In [ ]:
def get_buildings_for_style(soup, style):
    buildings=[]
   
    for building in soup.find_all('div',{'class': 'box'}):
        build_style = building.find('div',{'class': 'box_image'})['style']
        image_url = re.findall('\((.*?)\)', build_style)[0]
        build_dict={
            'Style':style,
            'Name':building.findChild('span',{'class':'title'}).text,
            'Bld_link':building.findChild('a').get_attribute_list(key='href')[0],
            'Image':image_url,
            'Address': str(building.findChild('div',{'class':'the_box_text'}).findChild('p')).replace('<br/>',' ').replace('<p>','').replace('</p>','')
        }

        download_image(style,image_url)
        #follow link to get more info on building
        if debug==True: 
            print(build_dict['Bld_link'])
        build_details_dict = get_building_details(build_dict['Bld_link'])
        build_dict = {**build_dict, **build_details_dict}
        buildings.append(build_dict)
   # bld_df = pd.DataFrame(buildings)
    return buildings

In [ ]:

def process_style (style):
    '''
    Test: #process_style('Arts and Crafts')
    '''
    print (f"loading {style}")
    curr_style_url = f"{style_url}?MainStyle={style}"
    soup=get_scrolling_page(curr_style_url)
    print(curr_style_url)
#     html = load_page(curr_style_url)
#     soup = BeautifulSoup(html)
#     if 'document.location' in soup.text:
#         # need to redirect to another page
#         redirect_url = soup.text.strip().replace('document.location = "','').replace('";',"")
#         curr_style_url=f"{site_root}{redirect_url}"
#        # print(curr_style_url)
#         html2 = load_page(curr_style_url)
#         soup = BeautifulSoup(html2)
        
    style_list = get_buildings_for_style(soup, style)
    print(style_list)
    buildings_list.extend(style_list)

In [ ]:
def main():
    '''
    run web scraping
    '''
    html = load_page(main_page)
    soup = BeautifulSoup(html)
    style_options = soup.find('select',{'name':'MainStyle'}).find_all('option')
    for style in style_options[1:]:
        process_style(style.text)
        time.sleep(5)
        bld_df = pd.DataFrame(buildings_list)
        bld_df.to_csv('../data/aco_buildings_'+ str(round(time.time(),0)) + '.csv')

In [13]:

if rerun_webscrape:
    main()
    bld_df = pd.DataFrame(buildings_list)
    bld_df.to_csv('../data/aco_buildings_'+ str(round(time.time(),0)) + '.csv')
else:
    # open file and save results to database
    bld_df=pd.read_csv('../data/aco_buildings_1543192082.0.csv',index_col=0) #, converters={"Companies": literal_eval}) #,  converters={1:ast.literal_eval})
    # clean up list stored in csv -- have to get python to treat as a list
    bld_df['Companies']=bld_df['Companies'].fillna('[]')
    bld_df.Companies = bld_df.Companies.apply(literal_eval)
    
bld_df.head()
db=connect_db() #establish connection
Session = sessionmaker(bind=db)
session = Session() 

In [ ]:
# bld_df['Companies']=bld_df['Companies'].fillna('[]')

In [ ]:
# bld_df.Companies = bld_df.Companies.apply(literal_eval)

In [ ]:
# bld_df.Companies

In [ ]:
# def save_to_database(db):
#     # recreate full links for urls
#     bld_df['Bld_link'] =bld_df['Bld_link'].apply(lambda x: f'{site_root}{x}' )
#     bld_df['Image'] =bld_df['Image'].apply(lambda x: f'{site_root[:-1]}{x}' )
#     db=connect_db() #establish connection
#     meta = MetaData(db)
#     table = Table('points_of_interest', meta, autoload=True)
    
#     for ix,row in bld_df.iterrows():
#         row_dict ={df_to_db_map[k]:v for k, v in row.items() if k in df_to_db_map.keys() and not pd.isnull(v)}
#         new_row=db.execute(table.insert(), [ 
#             row_dict
#         ])
#         new_id=new_row.inserted_primary_key[0]
#         if new_id:
#             db.execute('''INSERT INTO architectural_styles(poi_id, style) VALUES ( {},'{}')'''.format(new_id, row['Style']))
#             if row['Companies']:
#                 for architect in row['Companies']:

#                     db.execute('''INSERT INTO architects(poi_id, architect_name) VALUES ( {},'{}')'''.format(new_id, architect.replace("'","''")))


In [29]:
def save_to_database_ORM(session):
#     db=connect_db() #establish connection
#     Session = sessionmaker(bind=db)
#     session = Session() 
    for index, row in bld_df.iloc[754:-1,:].iterrows():
        
        poi_dict = {}
        poi_dict['name'] = row['Name']
        poi_dict['build_year'] = row['Completed']
        poi_dict['address'] = row['Address']
        poi_dict['external_url'] = row['Bld_link']

        poi = PointsOfInterest(**poi_dict )
        
        # define style
        style=ArchitecturalStyles(style=row['Style'])
        poi.styles.append(style)
        
        # architects (can be multiple)
        if row['Companies']:
            print(len(row['Companies']))
            prev_company=""
            for company in row['Companies']:
                print(company.replace("'","''"))
                if company != prev_company:
                    architect = Architects(architect_name= company.replace("'","''"))
                    poi.architects.append(architect)
                    prev_company=company
        session.add(poi)
        session.commit()

In [26]:
#session.rollback()

C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\orm\session.py:521: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  "Session's state has been changed on "


In [30]:
save_to_database_ORM(session)
#IntegrityError: duplicate key value violates unique constraint "architects_pkey"
#DETAIL:  Key (poi_id, architect_name)=(755, Moriyama and Teshima Architects) already exists.

2
Moriyama and Teshima Architects
Moriyama and Teshima Architects
1
Chadwick & Beckett
1
Peter Caspari
1
Peter Caspari
2
DunlopFarrow Architects Inc.
Burke, Horwood & White Associates
2
F. H. Herbert
Quadrangle Architects Ltd.
1
IBI Group
1
Isadore Feldman
2
ERA Architects Inc.
Neil G. Beggs
1
Stanley A. Waggett
1
Gordon & Helliwell
1
Herbert G. Paull
1
Hilditch Architect
2
John Anthony
Langley & Howland
1
Arthur W. Holmes
1
Robert Austin Abraham
1
G.W. Gouinlock
1
Frederick H. Herbert
1
Langley & Langley
1
R.J. Edwards
1
William L. Symons
1
Alfred Jordan Rattray
1
Henry Simpson
2
Bruce Kuwabara, Thomas Payne, Marianne McKenna and Shirley Blumberg
Kohn Shnier Architects
2
John Francis Brown
Josephine Young Architect
2
IBI Group
Page + Steele Inc.
2
Franklin E. Belfry
C.H. Bishop
2
Carrere & Hastings
Francis S. Baker
1
S. Hamilton Townsend
2
Stone Kohn Architects
John Richards
1
Darling and Pearson  Andrew Sharp was one of the architects working for this firm.
3
ERA Architects Inc.
Math

1
Strong & Voisy Architects
1
Strong Associates Architects
2
Kenneth R. Cooper  Often known as K.R. Cooper
Gordon S. Adamson Associates
1
WZMH Architects
1
Dietrick Boecker
1
Clark, Darling, Downey Architects
1
Adamson Associates
1
Kirshenblatt Korman Associates Architects & Planners
2
Adamson Associates
Moriyama and Teshima Architects
1
Moriyama and Teshima Architects
1
Adamson Associates
1
Adamson Associates
1
Pellow & Associates
1
Kohn Shnier Architects
1
Marani and Morris
1
Marani and Morris
1
David Sheppard and Powell
1
George N. Williams was the Chief Architect and the Deputy Minister of the Ontario Department of Public Works.
3
Murray Brown
George Martell Miller
Graziani + Corazza
2
Mathers and Haldenby
Diamond and Schmitt Architects
1
Kohn Shnier Architects
1
Charles Dolphin
1
C.H. Bishop
2
Ashley Woodburn
Melville J. Young
1
John M. Lyle
1
Page + Steele Inc.
1
Kirkor Architects and Planners
1
Page + Steele Inc.
1
Page + Steele Inc.
1
Burka Varacalli Architects
1
Burka Varacall

1
Common Architecture (Talayeh Hamidya)
1
Diamond and Schmitt Architects
1
architectsAlliance  Peter Clewes, Adrian DiCastri and John Van Nostrand are partners in this company.
1
architectsAlliance  Peter Clewes, Adrian DiCastri and John Van Nostrand are partners in this company.
1
architectsAlliance  Peter Clewes, Adrian DiCastri and John Van Nostrand are partners in this company.
1
Superkül
1
Kohn Pederson Fox
1
Kohn Pederson Fox
1
Wallman Architects  One of the principals of this firm is Rudy Wallman.
1
Core Architects Inc.
1
Core Architects Inc.
1
Core Architects Inc.
1
Shim Sutcliffe
1
Core Architects Inc.
1
Shim Sutcliffe
1
Core Architects Inc.
2
gh3
Maragna Architect Inc  The principal of this firm is Rocco Maragna.
1
Hariri Pontarini Architects
1
Diamond and Schmitt Architects
1
Teeple Architects Inc.
1
Kearns Mancini
1
Maclennan Jaunkalns Miller Architects  Also known as MJMA.
1
architectsAlliance  Peter Clewes, Adrian DiCastri and John Van Nostrand are partners in this compan

1
S. Hamilton Townsend
1
David Roberts
2
Molesworth, West & Secord
Gordon & Helliwell
1
Teeple Architects Inc.
1
Zak Ghanim was born in Egypt and came to Canada in 1975.
1
Zak Ghanim was born in Egypt and came to Canada in 1975.
1
Moriyama and Teshima Architects
1
Teeple Architects Inc.
2
Teeple Architects Inc.
Morphosis Inc.
1
Kirkor Architects and Planners
1
Kirkor Architects and Planners
2
Robbie/Young + Wright Architects
ALSOP Architects Ltd.
1
architectsAlliance  Peter Clewes, Adrian DiCastri and John Van Nostrand are partners in this company.
1
architectsAlliance  Peter Clewes, Adrian DiCastri and John Van Nostrand are partners in this company.
1
Teeple Architects Inc.
1
Quadrangle Architects Ltd.
1
Daniel Libeskind
2
W.W. Baldwin
Stone Kohn Architects
1
W.W. Baldwin
2
Henry Langley
Stone Kohn Architects
2
William Irving
John M. Lyle
1
Also see Douglas BrownÂ´s Book available in pdf. form in Resources
1
Sproatt and Rolph
1
J. M. Cowan
1
O.H. Fiureg
1
David Roberts
1
Walter R. Str

In [ ]:
# db_to_df_map={
#     'name':'Name',
#     'build_year': 'Completed'   ,
#     'demolished_year': 'Demolished' ,
#     'address':'Address'  ,
#     'external_url':  'Bld_link',
#     'details': 'Notes'
# }
# df_to_db_map={
#     'Name':'name',
#     'Completed':'build_year'   ,
#     'Demolished' :'demolished_year',
#     'Address' :'address' ,
#     'Bld_link':'external_url',
#     'Notes': 'details',
#     'Image':'image_url',
#     'Heritage':'heritage_stats',
#     'Current use':'current_use',
#     'Type':'poi_type'
# }

In [ ]:
# sql = """
# SELECT count(poi_id) FROM points_of_interest
# LIMIT 5""" 
# pd.read_sql(sql, con=db)

In [ ]:
# bld_df.shape

In [ ]:
# sql = """
# SELECT * FROM points_of_interest
# LEFT JOIN architects on points_of_interest.poi_id = architects.poi_id
# LEFT JOIN architectural_styles on points_of_interest.poi_id = architectural_styles.poi_id
# LIMIT 5""" 
# pd.read_sql(sql, con=db)

In [ ]:

# sql = """
# SELECT * FROM architectural_styles
# LIMIT 5""" 
# pd.read_sql(sql, con=db)

In [ ]:
# bld_df.head()

In [ ]:
# from sqlalchemy.orm import sessionmaker
# db=connect_db() #establish connection
# Session = sessionmaker(bind=db)
# session = Session() 


In [ ]:
# # Create 
# doctor_strange = Film(title="Doctor Strange", director="Scott Derrickson", year="2016")  
# session.add(doctor_strange)  
# session.commit()

# # Read
# films = session.query(Film)  
# for film in films:  
#     print(film.title)
# lBooks = DBSession.query(Book).filter(Book.price<20)
# # Update
# doctor_strange.title = "Some2016Film"  
# session.commit()

In [ ]:
# meta = MetaData(db)
# table = Table('points_of_interest', meta, autoload=True)



In [3]:
# from sqlalchemy import create_engine, Column, Integer, String, Sequence, Float
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy.orm import sessionmaker
# from models import connect_db, PointsOfInterest, ArchitecturalStyles, Architects,POICategories


In [4]:
# db=connect_db() #establish connection
# Session = sessionmaker(bind=db)
# session = Session() 

In [23]:
# bld_df2 = bld_df.copy()
# bld_df2 = bld_df2.iloc[29:30,:]
#bld_df.iloc[754:-1,:]

,Address,Bld_link,Companies,Completed,Current use,Demolished,Heritage,Image,Name,Notes,Style,Type
754,327 Bloor Street West University of Toronto To...,show_building.php?BuildingID=3422,"[Moriyama and Teshima Architects, Moriyama and...",1995,Cultural,NaN,No heritage status,/tobuilt_bk/php/Buildingimages/BataShoeMuseum1...,Bata Shoe Museum,The Bata Shoe Museum is a footwear museum esta...,Deconstructivism,Mid-rise
755,40 St. Leonard's Avenue Lawrence Park Toronto,show_building.php?BuildingID=7704,[Chadwick & Beckett],1910,Residential,NaN,No heritage status,/tobuilt_bk/php/Buildingimages/40StLeonards1.jpg,John McConnell House,Date is approximate.,Dutch colonial,Detached house
756,82 Buttonwood Avenue Mount Dennis York,show_building.php?BuildingID=8455,[],unknown,Health care,NaN,No heritage status,/tobuilt_bk/php/Buildingimages/WestParkRuddy1.jpg,West Park Healthcare Centre; Ruddy Building,NaN,Early modern,Mid-rise
757,600 Eglinton Avenue West Forest Hill Toronto,show_building.php?BuildingID=9434,[Peter Caspari],1954,Residential,NaN,No heritage status,/tobuilt_bk/php/Buildingimages/BuckinghamCourt...,Buckingham Court,NaN,Early modern,Mid-rise
758,660 Eglinton Avenue West Forest Hill Toronto,show_building.php?BuildingID=9435,[Peter Caspari],1953,Residential,NaN,No heritage status,/tobuilt_bk/php/Buildingimages/VincentCourt.jpg,Vincent Court,NaN,Early modern,Mid-rise
759,151 Front Street West Entertainment District T...,show_building.php?BuildingID=8575,[],1954,Office,NaN,Union Station Heritage Conservation District,/tobuilt_bk/php/Buildingimages/151FrontStW.jpg,Carrier Hotel and Data Centre,This building was constructed in 1954 to house...,Early modern,Mid-rise
760,33 Price Street Rosedale Toronto,show_building.php?BuildingID=8380,[],unknown,Residential,NaN,South Rosedale Heritage Conservation District,/tobuilt_bk/php/Buildingimages/33PriceSt1.jpg,33 Price Street,NaN,Early modern,Low-rise
761,573-575 Yonge Street Downtown Toronto,show_building.php?BuildingID=10658,[],1895,Commercial,NaN,Historic Yonge Street Heritage Conservation Di...,/tobuilt_bk/php/Buildingimages/582d08a6.jpg,573-575 Yonge Street,This is a early modern style building designed...,Early modern,Low-rise
762,574 Yonge Street Downtown Toronto,show_building.php?BuildingID=2512,[],unknown,Commercial,NaN,Historic Yonge Street Heritage Conservation Di...,/tobuilt_bk/php/Buildingimages/574Yonge.jpg,574 Yonge Street,This reconstructed building replaced part of w...,Early modern,Low-rise
763,619-623 Yonge Street Downtown Toronto,show_building.php?BuildingID=10669,[],1923,Commercial,NaN,Historic Yonge Street Heritage Conservation Di...,/tobuilt_bk/php/Buildingimages/d79bd836.jpg,619-623 Yonge Street,This is a 2 storey simple brick commercial bui...,Early modern,Low-rise


In [ ]:
#bld_df2['Companies'].astype('list')

In [ ]:
# poi_list = []
# for index, row in bld_df2.iterrows():
#     poi_dict = {}
#     poi_dict['name'] = row['Name']
#     poi_dict['build_year'] = row['Completed']
#     poi_dict['address'] = row['Address']
#     poi_dict['external_url'] = row['Bld_link']
#     poi_list.append(poi_dict)
    
    


In [ ]:
# #poi_list = []
# for index, row in bld_df2.iterrows():
#     poi_dict = {}
#     poi_dict['name'] = row['Name']
#     poi_dict['build_year'] = row['Completed']
#     poi_dict['address'] = row['Address']
#     poi_dict['external_url'] = row['Bld_link']
#     #poi_list.append(poi_dict)
#     poi = PointsOfInterest(**poi_dict )
#     # define style
#     style=ArchitecturalStyles(style=row['Style'])
#     poi.styles.append(style)
#     # architects (can be multiple)
#     if row['Companies']:
#         print(len(row['Companies']))
#         for company in row['Companies']:
#             print(company)#.replace("'","''"))
#             architect = Architects(architect_name= company)
# #             #db.execute('''INSERT INTO architects(poi_id, architect_name) VALUES ( {},'{}')'''.format(new_id, architect.replace("'","''")))
#             poi.architects.append(architect)
    
#     session.add(poi)
#     session.commit()

In [ ]:
# for poi in poi_list:
#     row = PointsOfInterest(**poi)
#     style=ArchitecturalStyles(style="my style")
#     architect = Architects(a)
#     row.styles.append(style)
#     session.add(row)


# session.commit()

In [ ]:
# >>>post=Post.query.get(post_id)
# comment=Comments(author='Leo',website='flask.com',content='flask-sqlalchemy example',post_id=post_id)
# >>> post.comments.append(comment)
# >>> db.session.add(post)
# >>> db.session.commit()

In [ ]:
# restaurant = Restaurant(restaurant_id = 'ABC123', price_range = 1)
# restaurant.categories.append(category)
# session.add(restaurant)
# session.commit()

In [ ]:
# row.poi_id

# new_id=new_row.inserted_primary_key[0]
#         if new_id:
#             db.execute('''INSERT INTO architectural_styles(poi_id, style) VALUES ( {},'{}')'''.format(new_id, row['Style']))
#             if row['Companies']:
#                 for architect in row['Companies']:

#                     db.execute('''INSERT INTO architects(poi_id, architect_name) VALUES ( {},'{}')'''.format(new_id, architect.replace("'","''")))


In [ ]:
for poi in session.query(PointsOfInterest).filter(PointsOfInterest.build_year=='1905'):
    print(poi.address)
    

In [18]:
df = pd.read_sql(session.query(PointsOfInterest).filter(PointsOfInterest.build_year=='1905').statement, session.bind)

In [ ]:
df

In [31]:
df = pd.read_sql(session.query(PointsOfInterest).statement, session.bind)

In [32]:
df

,poi_id,name,build_year,demolished_year,address,latitude,longitude,external_url,image_url,heritage_status,current_use,poi_type,source,details
0,1,Robert Grieg House,1905,None,20 Chestnut Park Rosedale Toronto,None,None,show_building.php?BuildingID=4386,None,None,None,None,None,None
1,2,22 Chestnut Park,1905,None,22 Chestnut Park Rosedale Toronto,None,None,show_building.php?BuildingID=4388,None,None,None,None,None,None
2,3,154-156 Amelia Street,unknown,None,154-156 Amelia Street Cabbagetown Toronto,None,None,show_building.php?BuildingID=3883,None,None,None,None,None,None
3,4,Sunny View Public School,1953,None,450 Blythwood Road Lawrence Park Toronto,None,None,show_building.php?BuildingID=11142,None,None,None,None,None,None
4,5,St. Matthew's Lawn Bowling Club,1906,None,450 Broadview Avenue Riverdale Toronto,None,None,show_building.php?BuildingID=3458,None,None,None,None,None,None
5,6,43 Castle Frank Crescent,1911,None,43 Castle Frank Crescent Rosedale Toronto,None,None,show_building.php?BuildingID=8092,None,None,None,None,None,None
6,7,Catherine Collard House,1909,None,584 Church Street Church-Wellesley Toronto,None,None,show_building.php?BuildingID=10625,None,None,None,None,None,None
7,8,43 Cross Street,unknown,None,43 Cross Street Weston York,None,None,show_building.php?BuildingID=8501,None,None,None,None,None,None
8,9,Alexander Gemmell House,1889,None,181 Ellis Avenue Swansea Toronto,None,None,show_building.php?BuildingID=7007,None,None,None,None,None,None
9,10,Herbert W. Waters House,1913,None,75 Elmer Avenue The Beach Toronto,None,None,show_building.php?BuildingID=9818,None,None,None,None,None,None


In [33]:
pd.read_sql(session.query(ArchitecturalStyles).statement, session.bind)

,poi_id,style
0,1,American colonial
1,2,American colonial
2,3,Arts and Crafts
3,4,Arts and Crafts
4,5,Arts and Crafts
5,6,Arts and Crafts
6,7,Arts and Crafts
7,8,Arts and Crafts
8,9,Arts and Crafts
9,10,Arts and Crafts


In [34]:
pd.read_sql(session.query(Architects).statement, session.bind)

,poi_id,architect_name
0,1,Alfred E. Boultbee
1,2,Alfred E. Boultbee
2,5,Robert McCallum
3,7,P. H. Finney
4,9,John Gemmell
5,28,Also see Douglas BrownÂ´s Book available in pd...
6,29,Charles J. Gibson
7,30,DunlopFarrow Architects Inc.
8,30,Lindsay and Brydon
9,37,Also see Douglas BrownÂ´s Book available in pd...
